<a href="https://colab.research.google.com/github/mariamandafm/ml-course-exercises/blob/main/Notebooks/employee_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Employee attrition predict

## Data

In [46]:
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
df = pd.read_csv('/content/drive/MyDrive/notebook_data/HR_Analytics.csv')
df.head()

,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,RM297,18,18-25,Yes,Travel_Rarely,230,Research & Development,3,3,Life Sciences,...,3,80,0,0,2,3,0,0,0,0.0
1,RM302,18,18-25,No,Travel_Rarely,812,Sales,10,3,Medical,...,1,80,0,0,2,3,0,0,0,0.0
2,RM458,18,18-25,Yes,Travel_Frequently,1306,Sales,5,3,Marketing,...,4,80,0,0,3,3,0,0,0,0.0
3,RM728,18,18-25,No,Non-Travel,287,Research & Development,5,2,Life Sciences,...,4,80,0,0,2,3,0,0,0,0.0
4,RM829,18,18-25,Yes,Non-Travel,247,Research & Development,8,1,Medical,...,4,80,0,0,0,3,0,0,0,0.0


In [124]:
# Convert to numeric format
df["Attrition"] = df["Attrition"].replace({'Yes': 1, 'No': 0})
df["AgeGroup"] = df["AgeGroup"].replace({'18-25': 0, '26-35': 1, '36-45': 2, '46-55': 3, '55+': 4})
df["BusinessTravel"] = df["BusinessTravel"].replace({'Non-Travel': 0, 'Travel_Rarely': 1, 'Travel_Frequently': 2, 'TravelRarely': 1})
df["Department"] = df["Department"].replace({'Research & Development': 0, 'Sales': 1, 'Human Resources': 2})
df["EducationField"] = df["EducationField"].replace({'Life Sciences': 0, 'Medical': 1, 'Marketing': 2,  'Technical Degree': 3, 'Human Resources': 4, 'Other': 5})
df["Gender"] = df["Gender"].replace({'Male': 0, 'Female': 1})
df["JobRole"] = df["JobRole"].replace(
    {'Sales Executive': 0, 'Research Scientist': 1, 'Laboratory Technician': 2,  'Manufacturing Director': 3, 'Healthcare Representative': 4, 'Manager': 5,  'Sales Representative': 6, 'Research Director': 7, 'Human Resources': 8}
  )
df["MaritalStatus"] = df["MaritalStatus"].replace({'Single': 0, 'Married': 1, 'Divorced': 2})

### Analises

In [30]:
df.groupby("AgeGroup")["Attrition"].mean()

AgeGroup
18-25    0.357724
26-35    0.189853
36-45    0.091295
46-55    0.118421
55+      0.170213
Name: Attrition, dtype: float64

In [33]:
df.groupby("MaritalStatus")["Attrition"].mean()

MaritalStatus
Divorced    0.103659
Married     0.123711
Single      0.253700
Name: Attrition, dtype: float64

In [36]:
df.groupby("BusinessTravel")["Attrition"].mean()

BusinessTravel
Non-Travel           0.079470
TravelRarely         0.000000
Travel_Frequently    0.247312
Travel_Rarely        0.150672
Name: Attrition, dtype: float64

In [9]:
df.groupby("Attrition")["DistanceFromHome"].mean()
df.groupby("Attrition").mean()

<ipython-input-9-1b587ff3cf0a>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby("Attrition").mean()


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Attrition,,,,,,,,,,,,,,,,,,,,,
No,37.538647,811.435588,8.935588,2.924316,1.0,1035.142512,2.772947,65.918680,2.770531,2.145733,...,2.730274,80.0,0.842190,11.854267,2.830918,2.780193,7.371176,4.483092,2.227858,4.373009
Yes,33.676471,748.932773,10.705882,2.840336,1.0,1014.735294,2.470588,65.462185,2.516807,1.642857,...,2.596639,80.0,0.529412,8.294118,2.626050,2.659664,5.121849,2.899160,1.945378,2.795652


In [14]:
df["Department"].value_counts()
df.groupby("Department")[]

Research & Development    967
Sales                     450
Human Resources            63
Name: Department, dtype: int64

## Decision tree

In [89]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

import joblib

In [79]:
feature_cols = ["AgeGroup", "BusinessTravel", "DailyRate", "Department", "DistanceFromHome", "Education", "EducationField", "Gender", "HourlyRate", "JobLevel", "JobRole", "MaritalStatus"]
X = df[feature_cols]
y = df["Attrition"]

In [80]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [111]:
# # Build decision tree model
# clf = DecisionTreeClassifier()
# # Train
# clf = clf.fit(X_train,y_train)
# y_pred = clf.predict(X_test)

In [82]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7522522522522522


In [84]:
!pip install graphviz pydotplus

In [112]:
# from sklearn.tree import export_graphviz
# from six import StringIO
# from IPython.display import Image
# import pydotplus

# dot_data = StringIO()
# export_graphviz(clf, out_file=dot_data,
#                 filled=True, rounded=True,
#                 special_characters=True,feature_names = feature_cols,class_names=['0','1'])
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
# graph.write_png('tree.png')
# Image(graph.create_png())

In [113]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

# Train
clf = clf.fit(X_train,y_train)
#Predict
y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8333333333333334


### Persist

In [90]:
# Persisting
joblib.dump(clf, 'employee.joblib')

['employee.joblib']

In [138]:
model = joblib.load('employee.joblib')
employee = df[feature_cols].iloc[99].values
predic = model.predict([employee])
predic[0]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


0

## App

In [176]:
# Retorna os funcionários da lista pode deixar a organização
employees = df.sample(n=20)
employees_data = employees[feature_cols].values
predictions = model.predict(employees_data)

for index, p in enumerate(predictions):
  if p == 1:
    print(employees["EmpID"].iloc[index])

RM1335


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
